In [1]:
import sys
!"{sys.executable}" -m pip install yahooquery
!"{sys.executable}" -m pip install scipy


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import io
import time
from scipy.stats import linregress

# Try to import yahooquery for fast filtering
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 0. HELPER FUNCTIONS (THE FIX)
# ==========================================
def calculate_z_score(info, fin, bs):
    """
    Calculates Altman Z-Score using Balance Sheet (bs) and Financials (fin).
    """
    try:
        # Get most recent data (column 0 is usually most recent in yfinance)
        total_assets = bs.loc['Total Assets'].iloc[0]
        total_liab = bs.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = bs.loc['Current Assets'].iloc[0]
        current_liab = bs.loc['Current Liabilities'].iloc[0]
        retained_earnings = bs.loc['Retained Earnings'].iloc[0]
        
        # EBIT might be labeled differently
        if 'EBIT' in fin.index:
            ebit = fin.loc['EBIT'].iloc[0]
        elif 'Operating Income' in fin.index:
            ebit = fin.loc['Operating Income'].iloc[0]
        else:
            ebit = 0
            
        total_revenue = fin.loc['Total Revenue'].iloc[0]
        market_cap = info.get('marketCap', 0)

        # Handle missing Market Cap by using Price * Shares if needed, 
        # but info['marketCap'] is usually reliable.
        if market_cap is None: return 0

        # Altman Z-Score Components
        # A: Working Capital / Total Assets
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings / Total Assets
        B = retained_earnings / total_assets
        
        # C: EBIT / Total Assets
        C = ebit / total_assets
        
        # D: Market Value of Equity / Total Liabilities
        D = market_cap / total_liab
        
        # E: Sales / Total Assets
        E = total_revenue / total_assets

        # Standard Formula for Manufacturing (General use)
        z = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z, 2)
    except Exception:
        return 0.0

def get_margin_trend(fin):
    """
    Determines if Operating Margin is Improving, Declining, or Stable.
    Handles ZeroDivisionError if Revenue is 0.
    """
    try:
        # Need at least 2 years of data
        if fin.shape[1] < 2: return "Stable"
        
        # Current Year
        op_inc_curr = fin.loc['Operating Income'].iloc[0]
        rev_curr = fin.loc['Total Revenue'].iloc[0]
        
        # Previous Year
        op_inc_prev = fin.loc['Operating Income'].iloc[1]
        rev_prev = fin.loc['Total Revenue'].iloc[1]

        # FIX: Check for Zero Revenue to avoid crash/warnings
        if rev_curr == 0 or rev_prev == 0:
            return "Unknown"

        margin_curr = op_inc_curr / rev_curr
        margin_prev = op_inc_prev / rev_prev

        # Logic: >5% improvement counts as "Improving"
        if margin_curr > (margin_prev * 1.05): return "Improving"
        elif margin_curr < (margin_prev * 0.95): return "Declining"
        else: return "Stable"
    except Exception:
        return "Unknown"

def get_interest_coverage(fin):
    """
    Calculates EBIT / Interest Expense.
    """
    try:
        if 'EBIT' in fin.index:
            ebit = fin.loc['EBIT'].iloc[0]
        else:
            ebit = fin.loc['Operating Income'].iloc[0]
            
        # Interest Expense is often negative in dataframes, use abs()
        # Some companies have 'Interest Expense' or 'Interest Expense Net'
        if 'Interest Expense' in fin.index:
            int_exp = abs(fin.loc['Interest Expense'].iloc[0])
        elif 'Interest Expense Net' in fin.index:
            int_exp = abs(fin.loc['Interest Expense Net'].iloc[0])
        else:
            int_exp = 0
        
        if int_exp == 0: return 100.0 # No debt interest is safe
        
        return round(ebit / int_exp, 2)
    except Exception:
        return 0.0

def calculate_roic(fin, bs):
    """
    Calculates Return on Invested Capital (ROIC).
    """
    try:
        # NOPAT (Net Operating Profit After Tax)
        if 'EBIT' in fin.index:
            ebit = fin.loc['EBIT'].iloc[0]
        else:
            ebit = fin.loc['Operating Income'].iloc[0]
            
        tax_exp = fin.loc['Tax Provision'].iloc[0]
        pre_tax_income = fin.loc['Pretax Income'].iloc[0]
        
        # Effective Tax Rate
        if pre_tax_income != 0:
            tax_rate = tax_exp / pre_tax_income
        else:
            tax_rate = 0.21 # Default fallback
            
        nopat = ebit * (1 - tax_rate)
        
        # Invested Capital = Total Assets - Non-Interest Bearing Current Liabs
        # Simplified: Total Assets - Current Liabilities
        total_assets = bs.loc['Total Assets'].iloc[0]
        curr_liab = bs.loc['Current Liabilities'].iloc[0]
        
        invested_capital = total_assets - curr_liab
        
        if invested_capital == 0: return 0.0
        
        return round((nopat / invested_capital) * 100, 2)
    except Exception:
        return 0.0

# ==========================================
# 2. STEP 1: FETCH UNIVERSE (Robust Version)
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    
    # 1. USA (NASDAQ Traded List)
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        # Filter out Test issues and ETFs
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        # Filter for length < 5 to avoid warrants/rights generally
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")

    # 2. CANADA (TSX Composite - Robust Search)
    try:
        url_ca = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
        headers = {"User-Agent": "Mozilla/5.0"} 
        r = requests.get(url_ca, headers=headers)
        
        # FIX: Scan ALL tables to find the one with 'Symbol'
        tables = pd.read_html(io.StringIO(r.text))
        df_ca = pd.DataFrame()
        
        for t in tables:
            if 'Symbol' in t.columns:
                df_ca = t
                break
        
        if not df_ca.empty:
            ca_list = df_ca['Symbol'].apply(lambda x: x.replace('.', '-') + ".TO").tolist()
            tickers.extend(ca_list)
            print(f"   -> Found {len(ca_list)} Canadian candidates.")
        else:
            print("   -> Warning: Could not find Canadian stock table on Wikipedia.")
            
    except Exception as e:
        print(f"   Error fetching Canada list: {e}")

    return list(set(tickers))

# ==========================================
# 3. STEP 2: SPLIT FILTER (With Debug Counts)
# ==========================================
def filter_universe_split(ticker_list):
    print(f"\n--- STEP 2: Applying Advanced Filters (YahooQuery) ---")
    
    # Configuration
    MIN_PRICE = 5.0
    MIN_CAP_US = 300_000_000
    MIN_VOL_US = 1_000_000
    MIN_CAP_CA = 100_000_000
    MIN_VOL_CA = 100_000
    
    MIN_CURRENT_RATIO = 1.0
    MIN_OP_MARGIN = 0.001
    MAX_PE = 100.0
    MAX_BETA = 3.0
    
    valid_candidates = []
    chunk_size = 500 
    
    # Counters for debugging
    cnt_us = 0
    cnt_ca = 0
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("summaryDetail defaultKeyStatistics price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # Extract
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                beta = summ_mod.get('beta', 0) or 0
                pe_ratio = summ_mod.get('trailingPE', 0)
                
                fin_mod = data[symbol].get('financialData', {})
                curr_ratio = fin_mod.get('currentRatio', 0) or 0
                op_margin = fin_mod.get('operatingMargins', 0) or 0
                
                # Logic
                if curr_price < MIN_PRICE: continue
                if curr_ratio < MIN_CURRENT_RATIO: continue
                if op_margin < MIN_OP_MARGIN: continue
                
                is_canada = symbol.endswith('.TO')
                
                if is_canada:
                    if mkt_cap < MIN_CAP_CA: continue
                    if avg_vol < MIN_VOL_CA: continue
                else:
                    if mkt_cap < MIN_CAP_US: continue
                    if avg_vol < MIN_VOL_US: continue

                if MAX_BETA and beta > MAX_BETA: continue
                if MAX_PE and pe_ratio and pe_ratio > MAX_PE: continue

                # Add to list
                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Market_Cap': mkt_cap,
                    'Region': 'Canada' if is_canada else 'USA',
                    'Current_Ratio': curr_ratio,
                    'Op_Margin': op_margin
                })
                
                # Update Counters
                if is_canada: cnt_ca += 1
                else: cnt_us += 1
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    print(f"      - USA Survivors: {cnt_us}")
    print(f"      - Canada Survivors: {cnt_ca}")
    
    return df

# ==========================================
# 4. STEP 3: DEEP DIVE (Credit Model)
# ==========================================
def run_credit_model(candidates_df):
    if candidates_df.empty: return None, None, None
    
    print(f"\n--- STEP 3: Deep Analysis (Credit Model + Options Check) ---")
    
    fortress, moonshot, distress = [], [], []
    
    for index, row in candidates_df.iterrows():
        ticker = row['Ticker']
        
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(candidates_df)}...", end='\r')
            time.sleep(0.2)
            
        try:
            stock = yf.Ticker(ticker)
            
            # 1. Option Check (Reliable)
            try:
                if not stock.options: continue 
            except:
                continue 
            
            # 2. Data
            info = stock.info
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            if 'Financial' in info.get('sector', ''): continue
            
            # 3. Metrics (Now calling the defined functions!)
            z = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            item = {
                'Ticker': ticker,
                'Price': row['Price'],
                'Region': row['Region'],
                'Z-Score': z,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Current_Ratio': row['Current_Ratio'],
                'Op_Margin': row['Op_Margin']
            }
            
            # 4. Buckets
            # Fortress: Strong Z-score, Stable/Improving margins, Good coverage, High ROIC
            if (z > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress.append(item)
            # Distress: Low Z-score or bad coverage
            elif (z < 1.8) or (int_cov < 1.5):
                distress.append(item)
            # Moonshot: Risky (low Z) but improving margins
            elif (z < 2.5) and (trend == "Improving"):
                moonshot.append(item)
                
        except:
            continue
            
    return pd.DataFrame(fortress), pd.DataFrame(moonshot), pd.DataFrame(distress)

# --- MAIN EXECUTION ---
# 1. Get Universe
raw_tickers = get_raw_universe()

# 2. Split Filter (US vs Canada)
filtered_df = filter_universe_split(raw_tickers)

# 3. Run Analysis
if not filtered_df.empty:
    fortress_df, moonshot_df, distress_df = run_credit_model(filtered_df)
    
    if fortress_df is not None and not fortress_df.empty:
        print("\n\n--- FORTRESS STOCKS (Top Picks) ---")
        # Check if running in a notebook (display) or script (print)
        try:
            display(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
        except NameError:
            print(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
    
    if moonshot_df is not None and not moonshot_df.empty:
        print("\n--- MOONSHOT STOCKS ---")
        try:
            display(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))
        except NameError:
            print(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.
   -> Warning: Could not find Canadian stock table on Wikipedia.

--- STEP 2: Applying Advanced Filters (YahooQuery) ---
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 541
      - USA Survivors: 541
      - Canada Survivors: 0

--- STEP 3: Deep Analysis (Credit Model + Options Check) ---
   Analyzing 540/541...

--- FORTRESS STOCKS (Top Picks) ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
103,NVDA,190.53,USA,91.42,Improving,341.19,78.13,4.468,0.63169
27,ISRG,577.81,USA,57.85,Improving,100.00,12.08,4.728,0.30326
61,RGLD,233.32,USA,46.35,Improving,44.71,10.28,3.516,0.50526
26,ORLA,14.60,USA,34.62,Improving,28.60,16.86,1.066,0.39538
112,APP,714.23,USA,32.60,Improving,5.95,39.46,3.250,0.76795
105,FAST,41.56,USA,30.86,Stable,207.59,28.83,4.259,0.20696
12,DOCS,43.69,USA,29.31,Improving,100.00,17.58,7.786,0.38546
117,RMBS,94.11,USA,28.62,Improving,142.27,14.36,11.609,0.35434
57,GOOG,314.96,USA,20.84,Improving,448.07,27.79,1.747,0.30512
36,VEEV,224.65,USA,16.85,Improving,100.00,9.04,7.527,0.29690



--- MOONSHOT STOCKS ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
7,NWS,29.87,USA,2.49,Improving,NaN,5.20,1.763,0.10448
1,NRG,160.88,USA,2.42,Improving,3.22,10.72,1.053,0.05527
17,HAYW,15.94,USA,2.42,Improving,3.12,6.52,3.184,0.16915
0,FTI,44.65,USA,2.37,Improving,10.66,18.86,1.130,0.15299
16,ACM,97.40,USA,2.37,Improving,5.97,13.63,1.135,0.07020
11,VRRM,22.50,USA,2.36,Improving,2.07,4.30,2.448,0.28560
23,NWSA,26.22,USA,2.35,Improving,NaN,5.20,1.763,0.10448
6,SHEL,72.81,USA,2.31,Improving,7.16,6.56,1.352,0.11347
20,GTES,21.97,USA,2.30,Improving,3.47,5.11,3.523,0.14935
13,HMY,21.74,USA,2.25,Improving,15.79,17.47,1.720,0.29669
